In [ ]:
# Necessary Packages 
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Create a ChromeOptions object to modify the browser settings
chrome_options = Options()

# Disable location access (block websites from accessing location)
prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the Chrome WebDriver with the modified options
driver = webdriver.Chrome(options=chrome_options)

# Starting URL
base_url = "https://careerviet.vn/viec-lam/tat-ca-viec-lam-vi.html"
driver.get(base_url)
time.sleep(random.randint(5, 10))

job_links_set = set([job.get_attribute('href') for job in driver.find_elements(By.CSS_SELECTOR, '.job_link')])

# Chuyển đổi set sang danh sách để có thứ tự
job_links = list(job_links_set)






# Lặp qua từng đường dẫn và truy cập vào trang chi tiết
for job_link in job_links:
    try:
        # Click vào đường dẫn để vào trang chi tiết
        driver.get(job_link)
        time.sleep(random.randint(2, 5))  # Đợi trang chi tiết tải
        
        # Lấy tiêu đề từ h1.title
        try:
            job_name = driver.find_element(By.CSS_SELECTOR, 'h1.title').text.strip()
        except:
            job_name = "Not specified"
        
        # Lấy tên công ty từ <a> với các lớp "employer job-company-name"
        try:
            company_name = driver.find_element(By.CSS_SELECTOR, 'a.employer.job-company-name').text.strip()
        except:
            company_name = "Not specified"
        
        # Lấy thông tin lương từ p tag sau strong với label "Lương"
        try:
            salary_info = driver.find_element(By.XPATH, "//strong[contains(text(),'Lương')]/following-sibling::p").text.strip()
        except:
            salary_info = "Not specified"
        
        # Lấy thông tin địa điểm
        try:
            location_elements = driver.find_elements(By.CSS_SELECTOR, '.map p a')
            locations = ', '.join([loc.text.strip() for loc in location_elements]) if location_elements else "Not specified"
        except:
            locations = "Not specified"
            
        # Lấy số năm kinh nghiệm
        try:
            experience = driver.find_element(By.XPATH, "//strong[contains(text(),'Kinh nghiệm')]/following-sibling::p").text.strip()
        except:
            experience = "Not specified"
            
        # Lấy cấp bậc
        try:
            job_level = driver.find_element(By.XPATH, "//strong[contains(text(),'Cấp bậc')]/following-sibling::p").text.strip()
        except:
            job_level = "Not specified"
            
        # Lấy ngày hết hạn nộp hồ sơ
        try:
            deadline = driver.find_element(By.XPATH, "//strong[contains(text(),'Hết hạn nộp')]/following-sibling::p").text.strip()
        except:
            deadline = "Not specified"
        
        # Lấy ngày cập nhật
        try:
            update_date = driver.find_element(By.XPATH, "//strong[contains(text(),'Ngày cập nhật')]/following-sibling::p").text.strip()
        except:
            update_date = "Not specified"
        
        # Lấy hình thức công việc
        try:
            employment_type = driver.find_element(By.XPATH, "//strong[contains(text(),'Hình thức')]/following-sibling::p").text.strip()
        except:
            employment_type = "Not specified"
        
        # Lấy thông tin ngành nghề mà chưa được
        try:
            industry_elements = driver.find_elements(By.CSS_SELECTOR, ".mdi-briefcase ~ p a")
            industries = ', '.join([industry.text.strip() for industry in industry_elements]) if industry_elements else "Not specified"
        except:
            industries = "Not specified"
            
        # Lấy danh sách phúc lợi
        try:
            welfare_elements = driver.find_elements(By.CSS_SELECTOR, "ul.welfare-list li")
            welfare_list = ', '.join([welfare.text.strip() for welfare in welfare_elements if welfare.text.strip()])
        except:
            welfare_list = "Not specified"
        
        # Lấy phần "Mô tả Công việc"
        try:
            # Tìm phần tử <div> có chứa phần "Mô tả Công việc"
            job_description_section = driver.find_element(By.XPATH, "//h2[contains(text(),'Mô tả Công việc')]/following-sibling::ul")
            # Tìm tất cả các phần tử <li> bên trong <ul> của "Mô tả Công việc"
            description_items = job_description_section.find_elements(By.TAG_NAME, "li")
            job_description = '\n'.join([item.text.strip() for item in description_items if item.text.strip()])
        except:
            job_description = "Not specified"
            
        # Lấy phần "Yêu Cầu Công Việc" 
        try:
            job_requirements_section = driver.find_element(By.XPATH, "//h2[contains(text(),'Yêu Cầu Công Việc')]/following-sibling::ul")
            requirements_items = job_requirements_section.find_elements(By.TAG_NAME, "li")
            job_requirements = '\n'.join([item.text.strip() for item in requirements_items if item.text.strip()])
        except:
            job_requirements = "Not specified"
        
        # Lấy nội dung của phần "Thông tin khác"
        try:
            other_info_section = driver.find_element(By.XPATH, "//h3[contains(text(),'Thông tin khác')]/following-sibling::div/ul")
            other_info_items = other_info_section.find_elements(By.TAG_NAME, "li")
            other_info = '\n'.join([item.text.strip() for item in other_info_items if item.text.strip()])
        except:
            other_info = "Not specified"
            
        # Lấy danh sách các Job Tags / Skills
        try:
            # Tìm tất cả các phần tử <li> trong phần "Job Tags / Skills"
            job_tags_elements = driver.find_elements(By.CSS_SELECTOR, "div.job-tags ul li")
            
            # Lấy text của từng mục <li> và ghép lại thành chuỗi, ngăn cách bằng dấu phẩy
            job_tags = ', '.join([tag.text.strip() for tag in job_tags_elements if tag.text.strip()])
        except:
            job_tags = "Not specified"
            
        # Lấy danh sách các link công việc tương tự
        try:
            # Tìm tất cả các phần tử <a> có class "job_link" trong phần công việc tương tự
            similar_job_links = driver.find_elements(By.CSS_SELECTOR, "div.jobs-list a.job_link")
            
            # Lấy thuộc tính href của từng link và ghép lại thành danh sách
            job_links = [link.get_attribute("href") for link in similar_job_links if link.get_attribute("href")]
        except:
            job_links = []
        
        
        # Tìm và nhấp vào "Tổng quan công ty"
        try:
            company_overview_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Tổng quan công ty')]")
            company_overview_link.click()
            time.sleep(3)  # Đợi trang công ty tải
        except:
            print("Không tìm thấy link 'Tổng quan công ty'.")
        # Tìm và nhấp vào tên công ty để vào trang chi tiết của công ty
        try:
            company_name_link = driver.find_element(By.CSS_SELECTOR, "a.name")
            company_name_link.click()
            time.sleep(3)  # Đợi trang chi tiết công ty tải
        except:
            print("Không tìm thấy link tên công ty.")
        # Lấy thông tin công ty
        try:
            # Lấy địa điểm công ty
            company_address = driver.find_element(By.XPATH, "//strong[contains(text(),'Địa điểm')]/following-sibling::p").text
            # Lấy quy mô công ty
            company_size_element = driver.find_element(By.XPATH, "//li[span[contains(@class, 'mdi-account-supervisor')]]")
            company_size_text = company_size_element.text
            company_size = company_size_text.split("Quy mô công ty:")[-1].strip()  # Chỉ lấy phần sau "Quy mô công ty:"
        except:
            print("Không tìm thấy thông tin công ty.")


        print(f"{job_name} - Salary: {salary_info} - Locations: {locations} - Experience: {experience} - Job Level: {job_level} - Deadline: {deadline} - Update Date: {update_date} - Industries: {industries} - Employment Type: {employment_type} - Link: {job_link}")
        print(f"Company: {company_name} - Địa điểm công ty: {company_address} - {company_size}")
        print(f"Phúc lợi: {welfare_list}")
        print(f"Mô tả công việc:\n{job_description}")
        print(f"Yêu cầu công việc:\n{job_requirements}")
        print(f"Thông tin khác:\n{other_info}")
        print(f"Job Tags / Skills: {job_tags}")
        print("Các link công việc tương tự:")
        for link in job_links:
            print(link)
        print("\n")

    except Exception as e:
        print(f"Error processing job at '{job_link}': {e}")

# Đóng trình duyệt
driver.quit()


Job link 1: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-phat-trien-van-hoa-doanh-nghiep.35C2207A.html
Job link 2: https://careerviet.vn/vi/tim-viec-lam/dai-dien-kinh-doanh-sr-can-tho-an-giang-kien-giang.35C2206E.html
Job link 3: https://careerviet.vn/vi/tim-viec-lam/hr-adm-supervisor.35C1734C.html
Job link 4: https://careerviet.vn/vi/tim-viec-lam/tuyen-dung-nhan-vien-sale-truc-page.35C176DB.html
Job link 5: https://careerviet.vn/vi/tim-viec-lam/tuyen-gap-vi-tri-tro-ly-giam-doc.35C17665.html
Job link 6: https://careerviet.vn/vi/tim-viec-lam/tro-giang-tieng-anh-nhon-trach-dong-nai.35C22072.html
Job link 7: https://careerviet.vn/vi/tim-viec-lam/giam-sat-kinh-doanh-ss-can-tho.35C2206C.html
Job link 8: https://careerviet.vn/vi/tim-viec-lam/talent-acquisition-specialist-urgent.35C1771C.html
Job link 9: https://careerviet.vn/vi/tim-viec-lam/bien-phien-dich-tieng-anh.35C177AB.html
Job link 10: https://careerviet.vn/vi/tim-viec-lam/ho-fdi-relationship-associate-officer-sme-division.35C177